In [1]:
#Bibliotecas de manipualção e visualização de dados
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
import os.path
from scipy.io.arff import loadarff 

from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

In [42]:
# Define tamanho padrão dos plotes
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10]

In [72]:
# Descomentar para rodar direto do collab
dataset_name = 'collins.arff'
if not os.path.isfile(dataset_name):
        urllib.request.urlretrieve("https://raw.githubusercontent.com/tadj8/mpd/main/datasets/collins.arff", dataset_name)

In [69]:
raw_data = loadarff("collins.arff")
df_data = pd.DataFrame(raw_data[0])

In [44]:
df_data

,Text,FirstPerson,InnerThinking,ThinkPositive,ThinkNegative,ThinkAhead,ThinkBack,Reasoning,Share_SocTies,Direct_Activity,...,SpaceInterval,Motion,PastEvents,TimeInterval,ShiftingEvents,Text_Coverage,Genre,Counter,Corpus,Corp.Genre
0,b'A01.TXT',0.09,1.72,0.62,0.70,2.15,0.66,1.23,2.24,0.48,...,0.75,0.35,2.55,1.63,0.79,28.31,b'1',1.0,b'1',b'101'
1,b'A02.TXT',0.13,1.48,0.38,0.63,1.85,0.93,1.48,2.28,0.21,...,0.59,0.25,1.77,1.39,0.80,23.91,b'1',2.0,b'1',b'101'
2,b'A03.TXT',0.04,1.72,0.34,1.12,1.55,0.90,1.68,2.84,0.17,...,0.65,0.17,2.32,1.94,0.82,30.15,b'1',3.0,b'1',b'101'
3,b'A04.TXT',0.00,2.74,0.90,2.43,1.21,1.03,3.46,3.06,0.22,...,0.67,0.09,1.89,1.71,0.81,29.43,b'1',4.0,b'1',b'101'
4,b'A05.TXT',0.26,2.15,0.39,0.61,2.28,0.75,1.93,1.84,0.44,...,0.75,0.35,1.54,1.80,0.96,27.89,b'1',5.0,b'1',b'101'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,b'rFR5.txt',0.49,3.25,0.41,2.10,1.07,0.25,3.29,1.93,1.65,...,0.70,0.90,1.19,1.77,1.07,34.11,b'15',996.0,b'2',b'215'
996,b'rFR6.txt',5.17,4.86,1.08,0.99,1.26,0.45,3.51,1.17,0.00,...,1.26,0.18,1.35,1.08,1.26,38.95,b'15',997.0,b'2',b'215'
997,b'rFR7.txt',0.00,2.70,1.30,1.21,1.21,1.30,2.16,1.35,0.18,...,1.08,0.31,5.13,1.71,1.08,40.36,b'15',998.0,b'2',b'215'
998,b'rFR8.txt',0.73,1.96,0.41,0.37,1.06,0.65,2.04,0.86,0.08,...,3.02,1.02,4.41,0.61,0.41,42.53,b'15',999.0,b'2',b'215'


In [62]:
df = df_data.copy()

# Nosso target:
#    Genre : The genre of each text (assigned by the Brown corpus compilers)

# Gênero é categórica e não tem ordinalidade entre as classes. Portanto, decidimos usar OneHotEncoding para transformar
# em numérica:
# df = df.join(pd.get_dummies(df['Genre'], prefix='genre'))
df['Genre'] = df['Genre'].factorize()[0]

# Remove colunas que não utilizaremos:
#   Genre : removemos coluna com valores categoricos
#   Text : nome do texto de origem
#   Corpus : 1=Brown, 2=Frown
#   Corp.Genre : mesmo que a coluna Genre, só codificada diferente
#
df.drop(['Text', 'Corpus', 'Counter', 'Corp.Genre'], axis=1, inplace=True)


In [63]:
df

,FirstPerson,InnerThinking,ThinkPositive,ThinkNegative,ThinkAhead,ThinkBack,Reasoning,Share_SocTies,Direct_Activity,Interacting,Notifying,LinearGuidance,WordPicture,SpaceInterval,Motion,PastEvents,TimeInterval,ShiftingEvents,Text_Coverage,Genre
0,0.09,1.72,0.62,0.70,2.15,0.66,1.23,2.24,0.48,0.00,2.11,5.23,5.01,0.75,0.35,2.55,1.63,0.79,28.31,0
1,0.13,1.48,0.38,0.63,1.85,0.93,1.48,2.28,0.21,0.17,1.77,3.46,4.34,0.59,0.25,1.77,1.39,0.80,23.91,0
2,0.04,1.72,0.34,1.12,1.55,0.90,1.68,2.84,0.17,0.17,2.84,5.03,5.85,0.65,0.17,2.32,1.94,0.82,30.15,0
3,0.00,2.74,0.90,2.43,1.21,1.03,3.46,3.06,0.22,0.31,2.47,3.15,3.28,0.67,0.09,1.89,1.71,0.81,29.43,0
4,0.26,2.15,0.39,0.61,2.28,0.75,1.93,1.84,0.44,0.35,2.54,5.79,3.16,0.75,0.35,1.54,1.80,0.96,27.89,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.49,3.25,0.41,2.10,1.07,0.25,3.29,1.93,1.65,2.39,3.17,4.28,4.20,0.70,0.90,1.19,1.77,1.07,34.11,14
996,5.17,4.86,1.08,0.99,1.26,0.45,3.51,1.17,0.00,0.58,2.92,4.72,7.11,1.26,0.18,1.35,1.08,1.26,38.95,14
997,0.00,2.70,1.30,1.21,1.21,1.30,2.16,1.35,0.18,0.04,2.25,10.61,6.74,1.08,0.31,5.13,1.71,1.08,40.36,14
998,0.73,1.96,0.41,0.37,1.06,0.65,2.04,0.86,0.08,1.18,1.47,11.23,11.02,3.02,1.02,4.41,0.61,0.41,42.53,14


In [64]:
#from pandas_profiling import ProfileReport
#profile = ProfileReport(df)
#profile.to_file('report_corpus.html')

Export report to file: 100%|██████████| 1/1 [00:00<00:00,  6.11it/s]
